In [4]:
import numpy as np
import pandas as pd
import math
import json
import random as rd
# setup data
def mySort(nums, inds):
    n = 1
    while n < len(nums):
        for i in range(len(nums) - n):
            if nums[i] < nums[i + 1]:
                nums[i], nums[i + 1] = nums[i + 1], nums[i]
                inds[i], inds[i + 1] = inds[i + 1], inds[i]
        n += 1
        
        
def fitness(data,dataSize, population, populationPrice, populationSize):
    sumWeight = 0
    sumCapacity = 0
    quality = []
    for i in range(0,populationSize): 
        for j in range(0,dataSize):
            if population[i][j] == 1:
                sumWeight += data[j][0]
                sumCapacity += data[j][1]
        if sumWeight > weight_carrying or sumCapacity > capacity:
            populationPrice[i] = 0
        sumWeight = 0
        sumCapacity = 0
    return populationPrice


    
raw_data = pd.read_csv('/Users/evgenijbuss/Downloads/3.txt', sep = ' ', names = ['weight','volume','price'], engine = 'python', header=None)
weight_carrying  = raw_data['weight'][0]
capacity = raw_data['volume'][0]
raw_data = raw_data.drop([0])
dataSize = raw_data.shape[0]
data = []
for i in range(0,dataSize):
    data.append((raw_data['weight'][i+1], raw_data['volume'][i+1], raw_data['price'][i+1]))
cheapestItem = raw_data['price'].min()    

    

# first population
firstPopulation = []
firstPopulationPrice = []
populationNum = 200
for j in range(0,populationNum):
    individual = []
    for i in range(0, dataSize):
        individual.append(0)
        startIndex = rd.randint(0, dataSize)
    sumWeight = 0
    sumCapacity = 0
    sumPrice = 0
    for i in range(startIndex,dataSize):
        if (sumWeight + data[i][0] <= weight_carrying) and (sumCapacity + data[i][1] <=capacity):
            individual[i] = 1
            sumWeight += data[i][0]
            sumCapacity += data[i][1]
            sumPrice += data[i][2]
    for i in range(0,startIndex):
        if (sumWeight + data[i][0] <= weight_carrying) and (sumCapacity + data[i][1] <=capacity):
            individual[i] = 1
            sumWeight += data[i][0]
            sumCapacity += data[i][1]
            sumPrice += data[i][2]
    firstPopulation.append(individual)
    firstPopulationPrice.append(sumPrice)
    sumPrice = 0
    
currentPopulation = firstPopulation
currentPopulationPrice = firstPopulationPrice
newBest = 0
count = 0






for p in range(0,500):
    #selection 
    currentPopulationPrice = fitness(data,dataSize,currentPopulation,currentPopulationPrice,len(currentPopulation))
    mySort(currentPopulationPrice,currentPopulation)
    currentBest = newBest
    newBest = currentPopulationPrice[0]
    sumWeight = 0
    sumCapacity = 0
    for j in range(0, dataSize):
            if currentPopulation[0][j] == 1:
                sumWeight += data[j][0]
                sumCapacity += data[j][1]
#     print(newBest,sumWeight,sumCapacity)


    if (newBest - currentBest == 0 and p > 50):
        count += 1
        if count == 10:
            print('Total price: ', newBest, '\n')
            print('Total weight: ',sumWeight, '\n')
            print('Total volume: ',sumCapacity, '\n')
            print('Items in data list:\n\n',currentPopulation[0])
            break



    selectionKf = populationNum * 0.2
    currentPopulationPriceSlice = currentPopulationPrice[0:int(selectionKf)]
    currentPopulationSlice = currentPopulation[0:int(selectionKf)]


    #crossingover
    usedIndexes = []
    children = np.zeros([int(selectionKf), dataSize], dtype = int)
    children = children.tolist()


    for i in range(0, int(selectionKf/2)):
        while(1):
            firstParentIndex = rd.randint(0, selectionKf - 1)
            secondParentIndex = rd.randint(0, selectionKf - 1)
            if (firstParentIndex != secondParentIndex) and (firstParentIndex not in usedIndexes) and (secondParentIndex not in usedIndexes):
                break
        for m in range(0, dataSize):
            children[i][m] = currentPopulationSlice[firstParentIndex][m] if rd.randint(0,1) == 0 else currentPopulationSlice[secondParentIndex][m]
            children[-i-1][m] = currentPopulationSlice[firstParentIndex][m] if rd.randint(0,1) == 0 else currentPopulationSlice[secondParentIndex][m]
        usedIndexes.append(firstParentIndex)
        usedIndexes.append(secondParentIndex)



    #mutation
    usedIndexes = []
    for i in range(0, int(len(children)*0.05)+1):
        while(1):
            randIndex = rd.randint(0, len(children)-1)
            if (randIndex not in usedIndexes):
                break
        while(1):
            randItem = rd.randint(0, dataSize - 1)
            if (children[randIndex][randItem] == 0):
                break
        children[randIndex][randItem] = 1

        
        
    #new population
    currentPopulation.reverse()
    currentPopulationPrice.reverse()
    childrenPrice = []
    sumPrice = 0
    for i in range(0, len(children)):
        sumPrice = 0
        for j in range(0, dataSize):
            if children[i][j] == 1:
                sumPrice += data[j][2]
        childrenPrice.append(sumPrice)
    mySort(childrenPrice, children)

    for i in range(0,int(selectionKf)):
        currentPopulation[i] = children[i]
        currentPopulationPrice[i] = childrenPrice[i]

# print(newBest)


Total price:  4328.0 

Total weight:  11701 

Total volume:  11.999999999999998 

Items in data list:

 [1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0]
